### Angel communities

In [1]:
import networkx as nx
import json
import numpy as np
import random
import scipy.special
import sys
import json
import pickle 
import argparse

from itertools import combinations
from tqdm import tqdm, trange
import sys

import re
import ast

In [2]:
# Load the graph
print('Reading the graph...')
G_directed = nx.read_gexf('../data_collection/data/GS.gexf')
print('Graph read.')

Reading the graph...
Graph read.


In [3]:
# Load the graph
print('Reading the graph...')
G = nx.read_gexf('../network_analysis/data/rpolitics/giant_component.gexf')
print('Graph read.')

Reading the graph...
Graph read.


In [4]:
def have_same_topic(G, u, v, w):
    u_submissions = re.findall('"(t3_.+?)"', G.nodes[u]['submissions'].replace("'",'\"'))
    v_submissions = re.findall('"(t3_.+?)"', G.nodes[v]['submissions'].replace("'",'\"'))
    w_submissions = re.findall('"(t3_.+?)"', G.nodes[w]['submissions'].replace("'",'\"'))
    u_t3 = set(u_submissions)
    v_t3 = set(v_submissions)
    w_t3 = set(w_submissions)
    
    if (u, v) in G.edges:
        uv_t3 = re.findall('"link_id": "(.+?)"', G.edges[u, v]['comments'].replace("'", '"'))
        u_t3.update(uv_t3)
        v_t3.update(uv_t3)
        
    if (v, u) in G.edges:
        vu_t3 = re.findall('"link_id": "(.+?)"', G.edges[v, u]['comments'].replace("'", '"'))
        v_t3.update(vu_t3)
        u_t3.update(vu_t3)
        
    if (v, w) in G.edges:
        vw_t3 = re.findall('"link_id": "(.+?)"', G.edges[v, w]['comments'].replace("'", '"'))
        v_t3.update(vw_t3)
        w_t3.update(vw_t3)
        
    if (w, v) in G.edges:
        wv_t3 = re.findall('"link_id": "(.+?)"', G.edges[w, v]['comments'].replace("'", '"'))
        v_t3.update(wv_t3)
        w_t3.update(wv_t3)
    
    return len(u_t3.intersection(v_t3.intersection(w_t3))) != 0

def topic_coverage(G):
    X = {}
    n_true = 0
    n = 0
    for (u, v) in list(G.edges()):
        for w in G.neighbors(u):
            if w == v: 
                continue
            # v -> u -> w 
            if not (v, w) in G.edges():
                if have_same_topic(G_directed, v, u, w):
                    n_true +=1
                n+=1
            
        for w in G.neighbors(v):
            if w == u: 
                continue
            # u -> v -> w 
            if not (w in X and X[w] == 1):
                if not (u, w) in G.edges():
                    if have_same_topic(G_directed, u, v, w):
                        n_true +=1
                    n+=1
    if n == 0:
        return 1, n, n_true
    return n_true / n, n, n_true

In [5]:
with open("./data/angel_communities.txt", "rb") as fp:
    communities = pickle.load(fp)

In [26]:
print(len(communities[11]))

6


In [25]:
res = 0
j = 0
for i in range(1, 12):
    subG = G.subgraph(communities[i])
    val, n, n_true = topic_coverage(subG)
    if n != 0:
        res += val * len(subG.nodes())
        j+=len(subG.nodes())
print(res / j)

0.6413979463159791
